In [1]:
import os
import conda
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(conda_dir, 'Library\share')
# proj_lib = os.path.join(os.path.join(conda_dir, 'pkgs'), 'proj4-5.2.0-h6538335_1006\Library\share')
path_gdal = os.path.join(proj_lib, 'gdal')
os.environ ['PROJ_LIB']=proj_lib
os.environ ['GDAL_DATA']=path_gdal

from datetime import datetime
import pandas as pd
import geopandas as gpd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon

In [119]:
buff_km = ['2','4','3']
poly_osmid = ['3368701','3338286','7700136']
place = ['Самара','Тольятти','Новокуйбышевск']
str_date = ['20200713_0959','20200713_1023','20200713_1026']

In [ ]:
# filename = './data/{}/{}/raw/layers/buildings_{}_{}_{}.shp'.format(poly_osmid[i], str_date[i],buff_km[i], place[i], str_date[i])

In [120]:
big_gdf = gpd.GeoDataFrame()
i = 0
for i in range(len(place)):
    filename = './data/{}/{}/raw/layers/buildings_{}_{}_{}.shp'.format(poly_osmid[i], 
                                                                       str_date[i], 
                                                                       buff_km[i], 
                                                                       place[i], str_date[i])
    gdf = gpd.read_file(filename, encoding='utf-8')
    gdf.crs = 'epsg:4326'
    gdf['city_name'] = place[i]
    gdf = gdf.to_crs('epsg:4326')
    big_gdf = big_gdf.append(gdf).reset_index(drop=True)
# 

In [123]:
big_gdf = big_gdf.drop_duplicates(subset=['geometry'])

In [ ]:
# np_bfgd = big_gdf.to_numpy()
# ind_oi = list(big_gdf.columns).index('osm_id')
# ind_ad = list(big_gdf.columns).index('address')

# lst_new_addr = []
# i=0
# for i in range(len(np_bfgd)):
#     one_addr = np_bfgd[i,ind_ad]
#     if (one_addr == "None, None") | ("None, " in one_addr):
#         newstr = None
#     elif ", None" in one_addr:
#         newstr = one_addr.replace(", None","")
#     else:
#         newstr = one_addr
#     lst_new_addr.append(newstr)
# # 
# big_gdf['address'] = lst_new_addr


In [ ]:
cp_bgdf = big_gdf.copy()
cp_bgdf = cp_bgdf.to_crs('epsg:32637')
np_cbfgd = cp_bgdf.to_numpy()
ind_oi = list(cp_bgdf.columns).index('osm_id')
ind_geo = list(cp_bgdf.columns).index('geometry')
ind_am = list(cp_bgdf.columns).index('amenity')
ind_bt = list(cp_bgdf.columns).index('bui_type')
ind_adr = list(cp_bgdf.columns).index('address')
ind_bl = list(cp_bgdf.columns).index('bui_lvl')

In [180]:
lst_liv = ['apartments', 'house', 'dormitory', 'hut', 'terrace']

lst_area = []
lst_bui_grp = []

i=0
for i in range(len(np_cbfgd)):
    p_area = np_cbfgd[i,ind_geo].area
    bui_type = np_cbfgd[i,ind_bt]
    amn = np_cbfgd[i,ind_am]
    addr = np_cbfgd[i,ind_adr]
    bui_lvl = np_cbfgd[i,ind_bl]
    
    if ((bui_type in lst_liv) | ((bui_type == 'yes') & (addr != None) & (amn == None))):
        bui_grp = "жилое"
    else:
        bui_grp = "нежилое"
    #
    if bui_lvl < 0:
        bui_lvl = abs(bui_lvl) + 1 # добавить наземный этаж
    #
    all_area = round(p_area * bui_lvl, 2)
    
    lst_bui_grp.append(bui_grp)
    lst_area.append(all_area)
# 

In [181]:
big_gdf['total_area_m'] = lst_area
big_gdf['bui_group'] = lst_bui_grp

In [ ]:
big_gdf.to_file('buildings_СТН.shp',encoding='utf-8')